In [24]:
from bloqade import move

Programs are represented by _kernels_, which can be generated by decorating functions with the `@move.vmove()` decorator to generate methods.

Lets write a kernel which includes every possible set of statements that you are allowed to use in this hackathon:

In [2]:
@move.vmove
def main():
    q = move.NewQubitRegister(5)

    state = move.Init(qubits=[q[0],q[1],q[2],q[3],q[4]], indices=[3,2,0,4,10])
    state.storage[[5, 6, 8, 9]] = move.Move(state.storage[[0, 1, 2, 3]])
    state.gate[[2,3]] = move.Move(state.storage[[2,9]])
    state.gate[[4,5]] = move.Move(state.gate[[2,3]])
    state.storage[[1,3]] = move.Move(state.gate[[4,5]])
    state = move.GlobalCZ(atom_state=state)
    state = move.GlobalXY(atom_state=state,x_exponent=1.0,axis_phase_exponent=1.0)
    state = move.LocalXY(atom_state=state,x_exponent=1.0,axis_phase_exponent=1.0,indices=[1,2])
    move.Execute(state)

When you execute this cell, it creates a method `main` which is a compiled representation of a program in Single Static Assignment (SSA) form. You can see the raw compiled code by calling the following. Each `%1` is an SSA value, which is a variable to be held in memory by the computer. These are generated by statements (such as e.g. `move.core.Constant(5)` which creates a constant integer SSA value).

The SSA code is not directly linked to the `main` function in the cell above because it has been _compiled_, lowering higher level statements to machine-level statements.

Lets go through each of these statements in more detail. Note that with an appropriate IDE, everything should be appropriately type hinted with documentation.

## Initialization

In [6]:
@move.vmove()
def make_new_register(n_qubits:int)->move.core.QubitRegister:
    return move.NewQubitRegister(n_qubits)

`move.NewQubitRegister` is a statement that creates a qubit register `QubitRegister` object within a kernel. When writing a main function, you will always need to declare these registers in order to instantiate atoms as qubits.

In [7]:
@move.vmove()
def initialize(register:move.core.QubitRegister) ->move.core.AtomState:
    return move.Init(qubits=[register[0],register[1],register[2]], indices=[1,4,2])

`move.Init` then places qubits within our two-zone register. The qubits, labeled by `register[i]` then get placed at locations in the memory region labeled by each index. This statement returns a `move.core.AtomState`, which tracks where each atom is in the register. In this example, qubits 0, 1, and 2 are put at locations 1, 4, and 2 in the storage zone of the register.

## Moves

In [8]:
@move.vmove()
def transfer_storage_to_gate(state:move.core.AtomState):
    state.gate[[3,4,5]] = move.Move(state.storage[[0,1,2]])
    return state


Next are "Transfer" statements, which move qubits around in the array. The syntax above represents two of the four possible transfers between the same zone, or between different zones. Here, we pick up whatever atoms happen to be in the storage region at indices `0`, `1`, and `2` with `move.Move(state.storage[[0,1,2]])`. Then, we move and put them back down in the gate region `state.gate[[3,4,5]] = *`. One can select the gate or storage region with `state.storage` and `state.gate` appropriately.

Our Acousto-Optical Deflector (AOD) constraints are enforced within these statements by enforcing that the indices are in increasing order. This enforces that the AODs never cross between the start and end!

`state.gate[[3,4,5]]` Good

`state.gate[[3]]` Good

`state.gate[[3,3]]` Bad (overlapping indices)

`state.gate[[3,2]]` Bad (out of order indices)

`state.gate[[5,4]] = move.Move(state.storage[[1,0]])` Bad, even though the AODs will not cross: each index selection are in decreasing order which is not allowed.

## Gates

In [9]:
@move.vmove()
def local_xy_rotation(state:move.core.AtomState):
    state = move.LocalXY(atom_state=state,x_exponent=1.0,axis_phase_exponent=1.0,indices=[3,4])
    return state

@move.vmove()
def local_z_rotation(state:move.core.AtomState):
    state = move.LocalRz(atom_state=state,phi=0.5,indices=[3])

A `move.LocalXY` statement executes 1 qubit rotation with the generator in the XY plane, e.g. $U=e^{i (cos(\phi) X + \sin(\phi) Y)\theta}$ where $\phi$=`axis_phase_exponent` and $\theta$=`x_exponent`.

Similarly, a `move.LocalRz` statement executes a 1 qubit Z phase rotation with the generator in the Z direction, eg $U=\hat Z^\theta$ where $\theta$=`exponent`.

These rotations are targeted on sites in the array, not qubits. The gate is acted in parallel on all qubits at register sites in the gate zone labeled by `indices`.

Note that this statement returns a new SSA value for the atom state, which has to be passed forward to future statements. This is in contrast to move statements, which mutate the atom state in place.

In [10]:
@move.vmove()
def global_xy_rotation(state:move.core.AtomState):
    state = move.GlobalXY(atom_state=state,x_exponent=1.0,axis_phase_exponent=1.0)
    return state

@move.vmove()
def global_z_rotation(state:move.core.AtomState):
    state = move.GlobalRz(atom_state=state,phi=0.5)
    return state

The local 1 qubit rotations have an equivalent global command, which acts on all qubits at the same time. *It is much better to do global rotations than local rotations*-- the lasers that execute the global rotations are much better calibrated, resulting in a higher fidelity execution. You will want to try to do as many global rotations as possible, and as few local rotations as possible.

In [11]:
@move.vmove()
def global_cz(state:move.core.AtomState):
    state = move.GlobalCZ(atom_state=state)
    return state

Then, there is the global CZ gate. This applies a CZ gate between adjacent pairs of atoms in the gate region all in parallel.

## A small technical command required by the compiler

In [12]:
@move.vmove()
def execute(state:move.core.AtomState):
    move.Execute(state)

It is important to declare this command at the end of your main program: otherwise the compiler will not "see" the fact that you have executed gates to affect the qubits and will ignore them. This is because the gate commands are "pure" meaning they do not mutate the input atom state but return a new atom state. This is different from the move commands, which do mutate the storage and gate regions that are attributes of the atom state.

## Scoring and analysis

Now lets put these statements together! We can write everything within a `main()` function. Lets just use the one we wrote above and do some analysis and animation!

All analysis for our challenge is within the `MoveScorer` class

In [ ]:
from iquhack_scoring import MoveScorer
analysis = MoveScorer(main,expected_qasm = "")

In [ ]:
# score:dict = analysis.score()
#for key,val in score.items():
#    print(f"{key}: {val}")

The lower the better on all of these! Your challenge, revealed tomorrow, will be scored based on these objectives.

In [ ]:
from bloqade.move.emit import MoveToQASM2
# Commented out due to bad rendering of qasm string
# analysis.validate_output(analysis.run_move_analysis())

# qasm = MoveToQASM2().emit_str(main)

Generate qasm strings to figure out what quantum circuit you just executed on hardware, and check if the circuit is actually what you were targeting. The target is input as a string to `MoveScorer` above. You won't score anything if the circuit you generate doesn't match the target of our challenge!

## Composition of Kernels

A useful thing that you can do is write kernels that can go inside of other kernels. We already did this to demonstrate each statement in the dialect, but lets demo this with a more formal builder pattern. For example, suppose we want to do a general 3 qubit unitary using a ZYZ decomposition; we can write the kernel as

In [16]:
@move.vmove()
def local_u3(state:move.core.AtomState, a:float, b:float, c:float, indices):
    state = move.LocalRz(atom_state=state, phi=a, indices=indices)
    state = move.LocalXY(atom_state=state, x_exponent=b,axis_phase_exponent=0.5, indices=indices)
    state = move.LocalRz(atom_state=state, phi=c, indices=indices)
    return state


@move.vmove()
def main2():
    q = move.NewQubitRegister(5)
    state = move.Init(qubits=[q[0],q[1],q[2],q[3],q[4]], indices=[3,2,0,4,10])
    state = local_u3(state, 0.1, 0.2, 0.3, [3,2])

Or, another common pattern is a builder function. Necessary variables in the global scope are pulled into the kernel's scope when declared, which lets us do things like the following:

In [17]:
def transfer_storage_to_gate(start:list[int],end:list[int]):
    @move.vmove()
    def kernel(state:move.core.AtomState):
        state.gate[end] = move.Move(state.storage[start])
        return state
    return kernel

move1 = transfer_storage_to_gate([0,1],[3,4])
move2 = transfer_storage_to_gate([2,3],[9,11])

@move.vmove()
def main3():
    q = move.NewQubitRegister(5)
    state = move.Init(qubits=[q[0],q[1],q[2],q[3],q[4]], indices=[3,2,0,4,10])
    move1(state)
    move2(state)
    

# SSA form

As an aside this might be a good time to discuss some details about the decorators you have been seeing. The new bloqade SDK uses an SSA-IR (Static Single Assignment Intermediate Representation) 
to represent programs. SSA-IR is used by must compilers to standardize analysis and optimization. For reading error messages it is useful to inspect the SSA form of the program specifically for 
moves as this has a lot of information packed into a single line of code. Take, for example, the transfer_storage_to_gate function:

```python
@move.vmove()
def transfer_storage_to_gate(state:move.core.AtomState):
    state.gate[[3,4,5]] = move.Move(state.storage[[0,1,2]])
    return state
```

Below is the SSA form of the program above:

```
func.func transfer_storage_to_gate(!py.AtomState) -> !Any {
  ^0(%transfer_storage_to_gate_self, %state):
  │ %0 = move.get_storage_zone(state=%state : !py.AtomState) : !py.StorageZone
  │ %1 = py.constant.constant IList([0, 1, 2]) : !Hinted(!py.IList, Value(data=IList([0, 1, 2])))
  │ %2 = move.get_sites(zone=%0, indices=%1) : !py.ZoneView[~L]
  │ %3 = move.move(sites=%2) : !py.MoveSites[~L]
  │ %4 = move.get_gate_zone(state=%state : !py.AtomState) : !py.GateZone
  │ %5 = py.constant.constant IList([3, 4, 5]) : !Hinted(!py.IList, Value(data=IList([3, 4, 5])))
  │ %6 = move.capture(zone=%4, indices=%5, sites=%3) : !py.Zone
  │      func.return %state : !py.AtomState
} // func.func transfer_storage_to_gate
```

lets examine the SSA form of the program in more line by line:

* `%0 = move.get_storage_zone(state=%state : !py.AtomState) : !py.StorageZone` corresponds to the python syntax `state.storage`. The `!py.AtomState` and `!py.StorageZone` are inferred types obtained via a _static_ analysis of the code. 

* `%1 = py.constant.constant IList([0, 1, 2]) : !Hinted(!py.IList, Value(data=IList([0, 1, 2])))` corresponds to the python syntax `[0, 1, 2]`. For performance reasons we stick to a more restrictive type of list called `IList` which is an immutable list that is more efficient to work with. The compiler can infer from the syntax that the expression is fully constant which is reflected in the `!Hinted(...)` type inference result

* `%2 = move.get_sites(zone=%0, indices=%1) : !py.ZoneView[~L]` corresponds to the  `storage[<key>]` syntax which returns a view of the zone.

* `%3 = move.move(sites=%2) : !py.MoveSites[~L]` is used to generate a new object that wraps the `ZoneView` object. This new object tells the compiler that the qubits within the view are going to be moved.

* `%4 = move.get_gate_zone(state=%state : !py.AtomState) : !py.GateZone` is the same as `%0` but for the gate region, e.g. `state.gate`.

* `%5 = py.constant.constant IList([3, 4, 5])` is the same as `%1` but for the gate region.

* `%6 = move.capture(zone=%4, indices=%5, sites=%3)` this final line is the most important statement, in python syntax it corresponds to: `gate[<key>] = <value>`. This syntax must have a "side effect", meaning the `gate` object is _*mutated*_ by this statement, e.g the compiler knows that the qubits are moving from the gate to the storage zone. 